### Import necessary libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)
import matplotlib.pyplot as plt
import seaborn as sns
import os

### Getting basic information

In [2]:
print(f"List all the file: {os.listdir()}")

List all the file: ['README.md', 'model_folder', 'exploratory_data_analytics.ipynb', 'gradient_boosting_model.ipynb', 'requirement.txt', 'model_testing', '.git', "[DSS-GROUP 1] Team-management - List of companies' data to be collected.csv"]


In [3]:
filepath = f"./[DSS-GROUP 1] Team-management - List of companies' data to be collected.csv"
with open(filepath, 'r') as opened_file:
    df = pd.read_csv(opened_file)

In [4]:
print("Original data's first 5 rows: ")
df.head()

Original data's first 5 rows: 


,Company name,Year,Quarter,TSR,PRASM,RASM,CASM,Load_factor,Gross_profit_margin,Quick_ratio,D/E,ROA,EPS
0,US_American Airlines,2013,1,0.0400,13.70,13.52,14.50,0.728,0.2600,0.73,2.23,0.001025,0.90
1,US_American Airlines,2013,2,0.1154,13.20,14.21,14.89,0.847,0.1130,0.71,4.43,0.009300,0.44
2,US_American Airlines,2013,3,0.0345,10.80,15.97,12.94,0.855,0.1621,0.47,4.10,0.012000,0.76
3,US_American Airlines,2013,4,0.0667,13.46,16.14,15.93,0.844,0.1800,0.52,3.82,0.008900,0.50
4,US_American Airlines,2014,1,0.0938,13.67,58.53,13.50,0.848,0.0731,1.01,0.35,0.011000,0.54


### Exploratory data analysis

In [5]:
print("Data' overall information:")
df.info()

Data' overall information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Company name         396 non-null    object 
 1   Year                 396 non-null    int64  
 2   Quarter              396 non-null    int64  
 3   TSR                  396 non-null    float64
 4   PRASM                396 non-null    float64
 5   RASM                 396 non-null    float64
 6   CASM                 396 non-null    float64
 7   Load_factor          396 non-null    float64
 8   Gross_profit_margin  396 non-null    float64
 9   Quick_ratio          396 non-null    float64
 10  D/E                  396 non-null    float64
 11  ROA                  396 non-null    float64
 12  EPS                  396 non-null    float64
dtypes: float64(10), int64(2), object(1)
memory usage: 40.3+ KB


1. Company name: Dimension of Location (Indvidual entity)
2. Year: Dimension of Time
3. Quarter: Dimension of Time

Sort data by the increasing dimension of time

In [6]:
df = df.sort_values(by=["Company name", "Year", "Quarter"])

### Feature engineering

In [7]:
df["Time"] = df["Year"] + (df["Quarter"]/4)
df = df.drop(["Year", "Quarter"], axis="columns")

In [8]:
df.columns
df.head()

,Company name,TSR,PRASM,RASM,CASM,Load_factor,Gross_profit_margin,Quick_ratio,D/E,ROA,EPS,Time
176,US_Alaska,0.161,11.50,14.01,0.1130,0.856,0.2384,1.11,0.700,0.0567,0.62,2013.25
177,US_Alaska,0.156,12.00,14.56,0.1140,0.850,0.2622,1.10,0.678,0.0620,0.75,2013.50
178,US_Alaska,0.096,12.50,15.10,0.1249,0.854,0.3065,1.14,0.690,0.0820,2.07,2013.75
179,US_Alaska,0.165,13.00,15.00,0.1302,0.856,0.2345,1.08,0.740,0.0876,1.11,2014.00
180,US_Alaska,0.181,13.82,14.50,0.1350,0.815,0.0769,1.07,0.740,0.0958,1.28,2014.25


In [9]:
df.columns

Index(['Company name', 'TSR', 'PRASM', 'RASM', 'CASM', 'Load_factor', 'Gross_profit_margin', 'Quick_ratio', 'D/E', 'ROA', 'EPS', 'Time'], dtype='object')

In [10]:
def engineered_time_impacted_features(df):
    column_list = [col for col in df.columns if col not in ["Company name", "Time", "TSR"]]
    for col in column_list:
        df[f"Engineered_{col}"] = df[col]*df["Time"]
    engineered_df = df.drop(column_list, axis="columns")
    return engineered_df

In [11]:
engineered_df = engineered_time_impacted_features(df)
engineered_df.head()

,Company name,TSR,Time,Engineered_PRASM,Engineered_RASM,Engineered_CASM,Engineered_Load_factor,Engineered_Gross_profit_margin,Engineered_Quick_ratio,Engineered_D/E,Engineered_ROA,Engineered_EPS
176,US_Alaska,0.161,2013.25,23152.375,28205.6325,227.497250,1723.34200,479.958800,2234.7075,1409.2750,114.151275,1248.2150
177,US_Alaska,0.156,2013.50,24162.000,29316.5600,229.539000,1711.47500,527.939700,2214.8500,1365.1530,124.837000,1510.1250
178,US_Alaska,0.096,2013.75,25171.875,30407.6250,251.517375,1719.74250,617.214375,2295.6750,1389.4875,165.127500,4168.4625
179,US_Alaska,0.165,2014.00,26182.000,30210.0000,262.222800,1723.98400,472.283000,2175.1200,1490.3600,176.426400,2235.5400
180,US_Alaska,0.181,2014.25,27836.935,29206.6250,271.923750,1641.61375,154.895825,2155.2475,1490.5450,192.965150,2578.2400


In [12]:
engineered_df.columns
lagged_features = [col for col in engineered_df.columns if col not in ["Company name", "Time"]]
print(lagged_features)

['TSR', 'Engineered_PRASM', 'Engineered_RASM', 'Engineered_CASM', 'Engineered_Load_factor', 'Engineered_Gross_profit_margin', 'Engineered_Quick_ratio', 'Engineered_D/E', 'Engineered_ROA', 'Engineered_EPS']


In [13]:
for feature in lagged_features:
    engineered_df[f'{feature}_lag1'] = engineered_df.groupby('Company name')[feature].shift(1)  # Lag1: Previous quarter for the same company
    engineered_df[f'{feature}_lag2'] = engineered_df.groupby('Company name')[feature].shift(2)  # Optional: Lag2: Two quarters ago for the same company


In [14]:
# rolling mean for each company, past 4 quarters
engineered_df['TSR_rolling_mean'] = engineered_df.groupby('Company name')['TSR'].rolling(window=4).mean().reset_index(0, drop=True)

In [15]:
# fill all the nulls value with the 0 value
engineered_df.fillna(0, inplace=True)

In [16]:
engineered_df.head()

,Company name,TSR,Time,Engineered_PRASM,Engineered_RASM,Engineered_CASM,Engineered_Load_factor,Engineered_Gross_profit_margin,Engineered_Quick_ratio,Engineered_D/E,Engineered_ROA,Engineered_EPS,TSR_lag1,TSR_lag2,Engineered_PRASM_lag1,Engineered_PRASM_lag2,Engineered_RASM_lag1,Engineered_RASM_lag2,Engineered_CASM_lag1,Engineered_CASM_lag2,Engineered_Load_factor_lag1,Engineered_Load_factor_lag2,Engineered_Gross_profit_margin_lag1,Engineered_Gross_profit_margin_lag2,Engineered_Quick_ratio_lag1,Engineered_Quick_ratio_lag2,Engineered_D/E_lag1,Engineered_D/E_lag2,Engineered_ROA_lag1,Engineered_ROA_lag2,Engineered_EPS_lag1,Engineered_EPS_lag2,TSR_rolling_mean
176,US_Alaska,0.161,2013.25,23152.375,28205.6325,227.497250,1723.34200,479.958800,2234.7075,1409.2750,114.151275,1248.2150,0.000,0.000,0.000,0.000,0.0000,0.0000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.0000,0.0000,0.0000
177,US_Alaska,0.156,2013.50,24162.000,29316.5600,229.539000,1711.47500,527.939700,2214.8500,1365.1530,124.837000,1510.1250,0.161,0.000,23152.375,0.000,28205.6325,0.0000,227.497250,0.000000,1723.3420,0.0000,479.958800,0.000000,2234.7075,0.0000,1409.2750,0.0000,114.151275,0.000000,1248.2150,0.0000,0.0000
178,US_Alaska,0.096,2013.75,25171.875,30407.6250,251.517375,1719.74250,617.214375,2295.6750,1389.4875,165.127500,4168.4625,0.156,0.161,24162.000,23152.375,29316.5600,28205.6325,229.539000,227.497250,1711.4750,1723.3420,527.939700,479.958800,2214.8500,2234.7075,1365.1530,1409.2750,124.837000,114.151275,1510.1250,1248.2150,0.0000
179,US_Alaska,0.165,2014.00,26182.000,30210.0000,262.222800,1723.98400,472.283000,2175.1200,1490.3600,176.426400,2235.5400,0.096,0.156,25171.875,24162.000,30407.6250,29316.5600,251.517375,229.539000,1719.7425,1711.4750,617.214375,527.939700,2295.6750,2214.8500,1389.4875,1365.1530,165.127500,124.837000,4168.4625,1510.1250,0.1445
180,US_Alaska,0.181,2014.25,27836.935,29206.6250,271.923750,1641.61375,154.895825,2155.2475,1490.5450,192.965150,2578.2400,0.165,0.096,26182.000,25171.875,30210.0000,30407.6250,262.222800,251.517375,1723.9840,1719.7425,472.283000,617.214375,2175.1200,2295.6750,1490.3600,1389.4875,176.426400,165.127500,2235.5400,4168.4625,0.1495


In [17]:
from sklearn.preprocessing import LabelEncoder

def label_encode(df):
    """
    Preprocess the DataFrame by encoding categorical variables before splitting into training and test sets.
    
    Parameters:
    - df: DataFrame containing the full dataset.
    
    Returns:
    - df_encoded: The DataFrame with categorical columns encoded.
    """
    
    # Identify categorical columns
    categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
    
    # Label Encoding for ordinal categorical variables or One-Hot Encoding for nominal ones
    df_encoded = df.copy()
    
    for col in categorical_columns:
        # If the column is a categorical feature, apply Label Encoding or One-Hot Encoding
        df_encoded[col] = df_encoded[col].astype(str)  # Ensure string type for encoding
        
        # Apply Label Encoding (you can use OneHotEncoder if needed for non-ordinal data)
        label_encoder = LabelEncoder()
        df_encoded[col] = label_encoder.fit_transform(df_encoded[col])
    
    return df_encoded

In [18]:
df_encoded = label_encode(engineered_df)

### Scaling data using Standard_Scaler

In [19]:
df_encoded.to_csv("final_data.csv", index=False)